# Last Background Checks

In [ ]:
import pandas as pd
import eleanor
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import numpy as np
import os
os.chdir('../scripts')
from analysis_tools_cython import mad_cuts,import_lightcurve

In [ ]:
main = pd.read_csv('../candidates/272-candidates.csv')
df = main[main.tags2 == 'red']

In [ ]:
df.head()

## Example 1

In [ ]:
tic_id = df.TIC_ID.tolist()
sector = df.Sector.tolist()

In [ ]:
star = eleanor.Source(tic=tic_id[0],sector=sector[0])
data = eleanor.TargetData(star,height=5,width=5)
vis = eleanor.Visualize(data)

In [ ]:
fig = vis.pixel_by_pixel(data_type='corrected',color_by_pixel=True)

---
## Lightkurve

In [1]:
import ipympl

In [2]:
import sys
sys.version

'3.6.6 |Anaconda, Inc.| (default, Oct  9 2018, 12:34:16) \n[GCC 7.3.0]'

In [3]:
#from bokeh.io import show, output_notebook
import lightkurve as lk
from astropy.table import Table

In [4]:
import bokeh
bokeh.__version__

'1.0.1'

In [5]:
tpf = lk.search_tesscut(f"TIC 237427836",sector=6).download_all(cutout_size=(15,15))
tpf = tpf[0]

In [12]:
tpf.interact()

In [11]:
%matplotlib inline

In [ ]:
custom_mask = tpf[0].create_threshold_mask(threshold=10)

raw_lc = tpf.to_lightcurve()
lc = tpf.to_lightcurve(aperture_mask=custom_mask).fill_gaps()

regressors = tpf.flux[:, ~custom_mask]
regressors.shape
bkg = np.median(regressors, axis=1)
bkg -= np.percentile(bkg, 5)


npix = custom_mask.sum()
median_subtracted_lc = lc - npix * bkg
raw_table = Table([raw_lc.time.value, raw_lc.flux, raw_lc.quality], names=['TIME', 'FLUX', 'QUALITY'])
raw_table = mad_cuts(raw_table,info=lc_info)
#raw_table = raw_table[raw_table['QUALITY']] == 0

table = Table([lc.time.value, lc.flux, lc.quality], names=['TIME', 'FLUX', 'QUALITY'])
table = mad_cuts(table,info=lc_info)
#table = table[table['QUALITY']] == 0

median_subtracted_table = Table([median_subtracted_lc.time.value, median_subtracted_lc.flux, median_subtracted_lc.quality], names=['TIME', 'FLUX', 'QUALITY'])
median_subtracted_table = mad_cuts(median_subtracted_table,info=lc_info)
#median_subtracted_table = median_subtracted_table[median_subtracted_table['QUALITY']] == 0


In [ ]:
closest_index = np.argmin(np.abs(table['TIME'] - 1486))
result = table[max(0, closest_index - 200): min(len(table), closest_index + 200)]



fig,ax = plt.subplots(1,2,figsize=(20,5))
#raw_table[raw_table.quality == 0].plot(ax=ax[0], normalize=True,linewidth=3,zorder=1,alpha=0.6,label='original')
#lc[lc.quality == 0].plot(ax=ax[0], normalize=True,linewidth=3,zorder=3,label='Simple Background Subtracted')

ax[0].plot(raw_table['FLUX']/np.nanmedian(raw_table['FLUX']),label='original')
ax[0].plot(table['FLUX']/np.nanmedian(table['FLUX'])-0.02,label='10 sigma mask')
ax[1].plot(table['TIME'],median_subtracted_table['FLUX']/np.nanmedian(median_subtracted_table['FLUX'])-0.02,label='10 sigma mask')
#median_subtracted_lc[lc.quality == 0].plot(ax=ax[1], normalize=True, label="Median Subtracted")
ax_top = ax[1].twiny()
ax_top.set_xlim(0, len(median_subtracted_lc.time))
ax_top.set_xlabel('Cadences')
ax[1].set_xlim([min(result['TIME'].value),max(result['TIME'].value)])


In [ ]:
tpf.animate(aperture_mask=custom_mask)

In [ ]:
tpf.animate(column="flux_bkg")

In [ ]:
tpf2 = lk.search_targetpixelfile("Beta Pictoris", mission='TESS', sector=6, cadence='long').download()

In [ ]:
tpf2.interact_sky() # does not work on JupyterLab -> related to Bokeh issue.

---

In [ ]:
from tqdm import tqdm

In [ ]:
result_dict = {}

for i in tqdm(df.reset_index().index):
    tic_id = df.iloc[i].TIC_ID
    sector = df.iloc[i].Sector
    transit_time = df.iloc[i].time
    
    # lightkurve TPF
    tpf = lk.search_tesscut(f"TIC {tic_id}", sector=sector).download_all(cutout_size=(15, 15))
    tpf = tpf[0]
    
    # lightkurve lc
    custom_mask = tpf[0].create_threshold_mask(threshold=10)
    raw_lc = tpf.to_lightcurve()
    lc = tpf.to_lightcurve(aperture_mask=custom_mask)
    
    fig, ax = plt.subplots(2, 1, figsize=(20, 15))
    ax[0].plot(raw_lc.flux / np.nanmedian(raw_lc.flux), label='original')
    ax[1].plot(lc.time.value[lc.quality == 0], lc.flux[lc.quality == 0] / np.nanmedian(lc.flux),
               label='10 sigma mask lightcurve')
    ax[1].axvline(transit_time, label='transit time from search')
    fig.legend()
    plt.close()
    animate = tpf.animate(aperture_mask=custom_mask)
    
    # Combine fig and animate into a single dictionary entry
    result_dict[tic_id] = {'fig': fig, 'animate': animate}

In [ ]:
with open('plots_and_tpf.pickle', 'wb') as file:
    pickle.dump(result_dict, file)

In [ ]:
dict_animate[354676567]

In [ ]:
dict_lc[354676567]

In [ ]:
df.TIC_ID

In [ ]:
classes = pd.DataFrame({"TIC_ID":df.TIC_ID})
classes['asteroid'] = ''

### Classes flags
- `0`: No asteroid
- `1`: Asteroid
- `2`: Did not plot
- `3`: Not sure

In [ ]:
classes.loc[271,'asteroid'] = 0

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [ ]:
classes.TIC_ID

In [ ]:
classes

In [ ]:
classes = classes.reset_index(drop=True)

In [ ]:
pd.merge(df

In [ ]:
test = pd.merge(main,classes,on='TIC_ID',how='outer')

In [ ]:
test.to_csv('../candidates/272-candidates.csv',index=None)

In [ ]:
test.loc[test.asteroid == 1, 'tags2'] = 'purple'

In [ ]:
to_export = test[test.asteroid == 0].abs_path.tolist()

In [ ]:
with open('47-candidates.txt', 'w') as file:
    for item in to_export:
        file.write(f"{item}\n")

In [ ]:
for_latex = test[(test.asteroid == 0) & (test.tags2 == 'red')]

In [ ]:
for_latex[['TIC_ID','Sector','snr','asym_score','skewness','abs_depth']].to_latex(index=None)

In [ ]:
for_latex.depth